In [5]:
# -*- coding: utf-8 -*-
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from deepface import DeepFace
import cv2
import glob
import pandas as pd
from tqdm import tqdm
import numpy as np
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows
from PIL import Image as PILImage
import io

INPUT_FOLDER = r"pictures"  
OUTPUT_EXCEL = r"pictures\face_high_precision_results_update.xlsx" 
SUPPORT_FORMATS = [".jpg", ".jpeg", ".png", ".bmp", ".tiff"]  
DETECTOR_BACKEND = "mtcnn"      # 检测器（mtcnn/retinaface/opencv）
EMOTION_CONF_THRESHOLD = 0.55   # 情绪置信度阈值
DETECT_TIMES = 3                # 单图检测次数（平衡精度/速度）
IMAGE_DISPLAY_SIZE = (150, 150) # Excel中图片显示尺寸

def preprocess_image(img_path):
    #图片预处理：增强对比度、归一化
    img = cv2.imread(img_path)
    if img is None:
        return None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_gray_clahe = clahe.apply(img_gray)
    img = cv2.cvtColor(img_gray_clahe, cv2.COLOR_GRAY2RGB)
    img = img / 255.0
    return img

def init_folders():
    os.makedirs(os.path.dirname(OUTPUT_EXCEL), exist_ok=True)
    os.makedirs(INPUT_FOLDER, exist_ok=True)

def insert_images_to_excel(excel_path, image_folder, image_size):
    wb = load_workbook(excel_path)
    ws = wb.active

    ws.column_dimensions['A'].width = 20   
    ws.column_dimensions['B'].width = 25    
    ws.column_dimensions['C'].width = 8     
    ws.column_dimensions['D'].width = 8    
    ws.column_dimensions['E'].width = 10    
    ws.column_dimensions['F'].width = 30   
    ws.column_dimensions['G'].width = 10  
    ws.column_dimensions['H'].width = 8     
    ws.column_dimensions['I'].width = 30   

    for row_idx in range(2, ws.max_row + 1):
        img_name = ws.cell(row=row_idx, column=1).value  
        if not img_name:
            continue

        img_path = os.path.join(image_folder, img_name)
        if not os.path.exists(img_path):
            print(f"图片 {img_name} 不存在")
            continue

        try:
            with PILImage.open(img_path) as img:
                img.thumbnail(image_size, PILImage.Resampling.LANCZOS)
                img_byte_arr = io.BytesIO()
                img.save(img_byte_arr, format='PNG')
                img_byte_arr.seek(0)

                excel_img = Image(img_byte_arr)
                excel_img.anchor = f'B{row_idx}' 

                ws.row_dimensions[row_idx].height = image_size[1] * 0.75  
                ws.add_image(excel_img)

        except Exception as e:
            print(f"嵌入图片 {img_name} 失败：{e}")
            continue

    wb.save(excel_path)


def main():
    init_folders()

    image_paths = []
    for file_name in os.listdir(INPUT_FOLDER):
        file_full_path = os.path.join(INPUT_FOLDER, file_name)
        if not os.path.isfile(file_full_path):
            continue
        
        file_ext = os.path.splitext(file_name)[1].lower()
        if file_ext in SUPPORT_FORMATS:
            image_paths.append(file_full_path)
    
    image_paths = list(set(image_paths))

    print(f"找到 {len(image_paths)} 张图片，开始识别...")
    results = []

    for img_path in tqdm(image_paths, desc="识别进度"):
        res = {
            "图片路径": img_path,        
            "文件名": os.path.basename(img_path),
            "年龄": None,
            "性别": None,
            "情绪": None,
            "情绪概率（详细）": None,
            "情绪置信度": None,
            "识别状态": "成功",
            "异常信息": None
        }

        try:
            preprocessed_img = preprocess_image(img_path)
            if preprocessed_img is None:
                raise Exception("图片损坏或无法读取")

            age_list, gender_list, emotion_list = [], [], []
            for _ in range(DETECT_TIMES):
                analyze_res = DeepFace.analyze(
                    img_path=img_path,
                    actions=["age", "gender", "emotion"],
                    detector_backend=DETECTOR_BACKEND,
                    align=True,
                    silent=True
                )
                single_res = analyze_res[0] if isinstance(analyze_res, list) else analyze_res
                age_list.append(single_res["age"])
                gender_list.append(single_res["dominant_gender"])
                emotion_list.append(single_res["emotion"])

            final_age = round(np.mean(age_list))
            final_age = max(1, min(final_age, 100))
            res["年龄"] = final_age

            final_gender = max(set(gender_list), key=gender_list.count)
            res["性别"] = final_gender

            emotion_probs = {}
            for emo in emotion_list[0].keys():
                emo_vals = [e[emo] for e in emotion_list]
                emotion_probs[emo] = round(np.mean(emo_vals), 3)
            dominant_emo = max(emotion_probs, key=emotion_probs.get)
            dominant_conf = emotion_probs[dominant_emo]
            if dominant_conf < EMOTION_CONF_THRESHOLD:
                dominant_emo = "不确定"
            res["情绪"] = dominant_emo
            res["情绪概率（详细）"] = str(emotion_probs)
            res["情绪置信度"] = dominant_conf

        except Exception as e:
            res["识别状态"] = "失败"
            res["异常信息"] = str(e)
            print(f"\n图片 {os.path.basename(img_path)} 识别失败：{e}")

        results.append(res)

    df = pd.DataFrame(results)
    df = df.drop(columns=["图片路径"])
    df = df.rename(columns={"文件名": "图片名称"})
    df["图片"] = ""

    column_order = [
        "图片名称",  "图片", "年龄", "性别", "情绪", 
        "情绪概率（详细）", "情绪置信度", "识别状态", "异常信息"
    ]
    df = df[column_order]

    df.to_excel(OUTPUT_EXCEL, index=False, engine="openpyxl")

    insert_images_to_excel(OUTPUT_EXCEL, INPUT_FOLDER, IMAGE_DISPLAY_SIZE)

    success_num = len(df[df["识别状态"] == "成功"])
    fail_num = len(df[df["识别状态"] == "失败"])
    print(f"\n识别结果保存至：{OUTPUT_EXCEL}")
    print(f"成功识别：{success_num} 张 | 识别失败：{fail_num} 张")
    if success_num > 0:
        emotion_certain_num = len(df[(df["识别状态"] == "成功") & (df["情绪"] != "不确定")])
        print(f"情绪高置信识别：{emotion_certain_num} 张（置信度≥{EMOTION_CONF_THRESHOLD}）")

if __name__ == "__main__":
    main()

找到 8 张图片，开始识别...


识别进度: 100%|██████████████████████████████████████████████████████████████████████████| 8/8 [01:02<00:00,  7.79s/it]



识别结果保存至：pictures\face_high_precision_results_update.xlsx
成功识别：8 张 | 识别失败：0 张
情绪高置信识别：8 张（置信度≥0.55）
